# ***Prediction models design***


## *Данный, второй документ в цикле статей описывает процесс выбора и обучения ML-модели для прогнозирования исходов теннисных матчей*


## Skills changelog:
*это первая попытка использования Machine Learning как инструмента в принципе, и поэтому
- есть только общее понимание того, что из себя представляет Data Science в целом и Machine Learning в частности;
- есть огромное желание попробовать ML на практике;
- используются любые руководства для начинающих;
- отсутствуют специфические знания и навыки, а именно - для подготовки данных, обучения моделей и т.п.;
- все, чем приходится пока руководствоваться - логика и здравый смысл.


## Sources:
- Python 3 and Pandas docs;
- StackOverflow


feature selection test

In [1]:
### !pip install pandas
### !pip install matplotlib
import pandas as pd
import numpy as np
#import urllib
import urllib.request
import os
import glob
import csv
import re
from datetime import datetime
from IPython.display import display
from pandas.plotting import scatter_matrix
import matplotlib

# Pandas display options tweaking
pd.options.display.max_columns = None
INIT_MAX_ROWS = 50
pd.options.display.max_rows = INIT_MAX_ROWS

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
print('Root directory:', ROOT_DIR)
# Downloaded files path
DOWNL_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_downloaded')
# Preprocessed files path
PREP_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_preprocessed')

Root directory: /home/vl/Repos/machine-learning-on-tennis


In [2]:
preprocessed_data_file = 'atp_matches_1991-2018_preprocessed_final.csv'
preprocessed_data_full_path = os.path.join(PREP_F_P, preprocessed_data_file)
preprocessed_data = pd.read_csv(preprocessed_data_full_path, header=0, encoding='utf-8-sig', engine='python')

display(preprocessed_data)

,ht,age,rank,rank_points,ace_last,ace_max,ace_mean,ace_min,df_last,df_max,df_mean,df_min,perc_1stIn_last,perc_1stIn_max,perc_1stIn_mean,perc_1stIn_min,perc_w_1stIn_last,perc_w_1stIn_max,perc_w_1stIn_mean,perc_w_1stIn_min,perc_w_2ndIn_last,perc_w_2ndIn_max,perc_w_2ndIn_mean,perc_w_2ndIn_min,winner_flag
0,-0.1000,-0.3333,0.0769,-0.0004,0.2000,-0.3333,0.0000,1.0000,0.0000,1.0000,1.0000,0.0000,-0.0909,-0.1667,-0.1250,-0.1429,-0.3333,0.0000,-0.2500,-0.1000,-0.0769,-0.3333,-0.1000,-0.0714,-1
1,0.1000,-0.2000,-0.0345,0.0002,0.0909,0.0909,0.1250,0.1667,-0.1250,-0.1429,-0.1667,-0.2500,0.1000,0.1250,0.1111,0.0588,0.0500,0.1111,0.0714,0.0625,0.0476,0.0833,0.0667,0.0625,1
2,0.3333,-0.5000,-0.0303,0.0007,-0.0625,-0.0833,-0.1000,-0.2000,-0.2000,-0.5000,-1.0000,-0.5000,0.0714,0.0625,0.0588,0.0556,-0.1111,-0.1250,-0.0909,-0.1111,0.1429,0.5000,0.1000,0.0714,1
3,0.3333,-0.1667,0.1250,-0.0003,-1.0000,-0.2500,-0.2500,-0.3333,0.5000,0.0000,0.0000,-1.0000,-0.1250,-0.1250,-0.2500,-0.3333,-0.3333,-0.0769,-0.1000,-0.1000,-0.0370,-0.1250,-0.0476,-0.0385,1
4,-0.0455,-0.1000,0.1111,-0.0003,-1.0000,-0.1667,-0.2000,-0.2500,1.0000,0.0000,0.0000,1.0000,0.2000,0.1667,0.2000,0.2000,-0.1111,-0.2500,-0.1111,-0.0714,-0.0435,-0.0526,-0.0625,-0.0625,-1
5,0.3333,1.0000,-0.0147,0.0007,-0.5000,0.3333,-1.0000,-0.3333,-0.3333,-1.0000,-0.5000,-0.3333,0.0417,0.0455,0.0769,0.2000,0.0000,0.0000,0.1250,0.0714,-0.1667,-0.1667,-0.2000,-0.3333,-1
6,0.1250,0.1667,-0.0303,0.0022,1.0000,-0.2000,-0.3333,-0.3333,0.0000,0.0000,-1.0000,-1.0000,0.0500,0.0909,0.0833,0.0769,-0.0833,-0.1111,-0.0909,-0.0833,-0.0357,-0.1429,-0.1000,-0.0667,1
7,-0.3333,0.1250,0.0400,-0.0013,-0.0333,-0.0417,-0.0588,-0.1667,1.0000,1.0000,-0.5000,-0.3333,-0.0667,-0.1429,-0.0769,-0.0667,-0.1667,-0.5000,-0.2000,-0.1667,1.0000,-0.1000,-0.0909,-0.0556,-1
8,-0.3333,0.1667,-0.0169,0.0001,0.0000,0.2000,0.5000,1.0000,-0.3333,-0.3333,-0.3333,-0.5000,-0.2000,-0.2000,1.0000,0.0714,-0.2000,0.3333,0.2500,0.2000,-0.3333,0.1429,0.0909,0.0526,1
9,-0.1000,0.5000,0.0625,-0.0003,-0.1429,-0.0769,-0.1429,-0.2500,-1.0000,0.5000,1.0000,-1.0000,-0.2500,0.2000,0.2500,0.1250,-0.1667,-0.2500,-0.2500,-0.2000,0.0714,0.2500,-0.2000,-0.2000,-1


In [ ]:
%time scatter_matrix(preprocessed_data, figsize=(400,400))

CPU times: user 1min 36s, sys: 1min 4s, total: 2min 41s
Wall time: 1min 11s
